In [1]:
import jinja2
from  anthropic import AnthropicBedrock
from compiler.core import Compiler
from tracing_client import TracingClient
from policies import common, handlers

In [2]:
jinja_env = jinja2.Environment()
compiler = Compiler("botbuild/app_schema", "botbuild/app_schema")
client = TracingClient(AnthropicBedrock(aws_profile="dev", aws_region="us-west-2"))

In [3]:
application_description = "Bot that always responds with a greeting."
application_description = "Bot that writes poems about cars."

In [4]:
import os

DATASET_DIR = "evals/dataset.min/"
files = os.listdir(DATASET_DIR)
drizzle_file = "car_poem_drizzle_schema.ts"
typescript_file = "car_poem_typescript_schema.ts"
with open(os.path.join(DATASET_DIR, drizzle_file), "r", encoding="utf-8") as f:
    drizzle_schema = f.read()
with open(os.path.join(DATASET_DIR, typescript_file), "r", encoding="utf-8") as f:
    typespec_definitions = f.read()

content = jinja_env.from_string(handlers.PROMPT).render(application_description=application_description,#
                                                        function_name="getCarPoem", 
                                                        drizzle_schema=drizzle_file, 
                                                        typespec_definitions=typespec_definitions)
message = {"role": "user", "content": content}

In [5]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    output = handlers.HandlerTaskNode.run([message], function_name="getCarPoem")

Langfuse client is disabled since no public_key was provided as a parameter or environment variable 'LANGFUSE_PUBLIC_KEY'. See our docs: https://langfuse.com/docs/sdk/python/low-level-sdk#initialize-client


In [6]:
output.output.handler


HandlerOutput(handler='import { db } from "../db";\nimport { carPoemTable } from "../db/schema/car_poem_drizzle_schema";\n\ninterface Options {\n    carId: number;\n}\n\nconst handle = async (options: Options): Promise<string> => {\n    const result = await db\n        .select()\n        .from(carPoemTable)\n        .where(eq(carPoemTable.carId, options.carId))\n        .limit(1);\n\n    if (result.length === 0) {\n        throw new Error(`No poem found for car ID: ${options.carId}`);\n    }\n\n    return result[0].poemText;\n};', feedback={'exit_code': 2, 'stdout': "src/handlers/dec46e48-39d4-4881-b48c-e04074da73c3.ts(2,30): error TS2307: Cannot find module '../db/schema/car_poem_drizzle_schema' or its corresponding type declarations.\nsrc/handlers/dec46e48-39d4-4881-b48c-e04074da73c3.ts(8,7): error TS6133: 'handle' is declared but its value is never read.\nsrc/handlers/dec46e48-39d4-4881-b48c-e04074da73c3.ts(12,16): error TS2304: Cannot find name 'eq'.\nsrc/handlers/dec46e48-39d4-488

In [7]:
root = handlers.HandlerTaskNode(output)

In [8]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    solution = common.bfs(root)

In [9]:
solution.score, solution.depth

(0, 0)

In [10]:
if solution.parent:
    for node in solution.parent.children:
      print(node.is_successful, node.data.output)

In [11]:
with handlers.HandlerTaskNode.platform(client, compiler, jinja_env):
    args = root.run_args

In [12]:
args

[{'role': 'assistant',
  'content': 'Based on the provided context, I\'ll generate a handler for the `getCarPoem` function that focuses on database operations. Since there\'s no TypeSpec definition provided, I\'ll assume this handler needs to generate or retrieve car-related poems from a database.\n\n<handler>\nimport { db } from "../db";\nimport { carPoems } from \'../db/schema/car_poem_drizzle_schema\';\nimport { eq } from \'drizzle-orm\';\n\ninterface Options {\n    carBrand?: string;\n    carModel?: string;\n}\n\nconst handle = async (options: Options): Promise<string> => {\n    // Try to find an existing poem first\n    const existingPoem = await db\n        .select()\n        .from(carPoems)\n        .where(\n            options.carModel \n                ? eq(carPoems.carModel, options.carModel)\n                : eq(carPoems.carBrand, options.carBrand!)\n        )\n        .limit(1);\n\n    // Return existing poem if found\n    if (existingPoem.length > 0) {\n        return exi